<a href="https://colab.research.google.com/github/AslauAlexandru/Techwithwarriors-Generative-AI-Internship-Tasks/blob/main/TECHWITHWARRIORS_Generative_AI_Internship_Task_05.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Task 05

Fine-tune a large language model for domain-
specific text generation or translation tasks.

Task Details: Customize a large language model to
generate text or perform translations tailored to a
specific domain.

In [1]:
!pip install transformers
!pip install datasets
!pip install peft
!pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.3/527.3 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 1.5 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 17.0.0 which is incompatible.
ibis-framework 8.0.0 requires pyarrow<16,>=2, but you have pyarrow 17.0.0 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 296.4/296.4 kB 5.3 MB

In [2]:
! pip install datasets evaluate transformers rouge-score nltk
!pip install huggingface_hub
!pip install accelerate

  Preparing metadata (setup.py) ... done
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24935 sha256=50d6d57b38a27bb8c3481048a5761c8060c4b8d42595f8d884b4e0d926c7c54f
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge-score


# Fine-tuning LLM for Text Generation

In [3]:
'''
import torch
from transformers import GPT2Tokenizer, GPT2LMHeadModel

#Load the pre-trained GPT-2 model and tokenizer
model_name='gpt2'
tokenizer=GPT2Tokenizer.from_pretrained(model_name)
model=GPT2LMHeadModel.from_pretrained(model_name)


num_params = sum(p.numel() for p in model.parameters())
print(f"Number of parameters in GPT-2 model: {num_params}")
#124.39 Million


#Set the device to GPU if available, else use CPU
device=torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

#Text generation parameters
prompt_text="I can do"
max_length=10
num_samples=1

#Generate text samples
generated_texts=[]
for _ in range(num_samples):
  input_ids = tokenizer(prompt_text, return_tensors="pt").input_ids.to(device)
  #input_ids = tokenizer.encode(prompt_text, return_tensor="pt").to(device)
  with torch.no_grad():
    output=model.generate(
         input_ids,
         max_length=max_length,
         pad_token_id=tokenizer.eos_token_id,
         num_return_sequences=1,
         temperature=0.7 #Controls the randomness of generated text
    )
    generated_text=tokenizer.decode(output[0], skip_special_tokens=True)
    generated_texts.append(generated_text)


#Print the generated text samples
for i, text in enumerate(generated_texts):
  print(f"Generated Text {i+1}: {text}")



## Fine-tuning LLM for Text Generation: GPT-2 model

#Replace with your text generation dataset
train_texts=[
    "My name is Li's Bot.",
    "I can do anything.",
    "I am trained for giving a answer or answers.",
    "I am here for helping you."
    #Add more training examples here
]

#Set the pad_token for the tokenizer
tokenizer.pad_token=tokenizer.eos_token

#Tokenize the training texts
train_encodings = tokenizer(train_texts, padding=True, truncation=True)
#train_encodings=tokenizer(train_texts, padding=True, truncation=True, return_tensor="pt")
#train_encodings.to(device)
# Convert to tensor and move to device (if using GPU)
train_encodings = torch.tensor(train_encodings["input_ids"]).to(device)


#Fine-tune the GPT-2 model for text generation
optimizer=torch.optim.AdamW(model.parameters(), lr=1e-5)
model.train()

epochs=300
for epoch in range(epochs):
  outputs = model(labels=train_encodings[:, 0], **train_encodings)  # Access first element for labels
  #outputs=model(train_encodings, labels=train_encodings.input_ids)
  loss=outputs.loss
  loss.backward()
  optimizer.step()
  optimizer.zero_grad()
  print(f"Epoch {epoch+1}-Loss {loss.item()}")

#Save the fine-tuned model
output_dir="/content/fine_tuned_model_gpt2"
model.save_pretained(output_dir)
tokenizer.save_pretained(output_dir)


# Replace with your text generation dataset
train_texts = [
    "My name is Li's Bot.",
    "I can do anything.",
    "I am trained for giving a answer or answers.",
    "I am here for helping you."
    # Add more training examples here
]

# Set the pad token for the tokenizer
tokenizer = GPT2Tokenizer.from_pretrained(model_name)  # Replace if using a different model
tokenizer.pad_token = tokenizer.eos_token

# Tokenize the training texts
train_encodings = tokenizer(train_texts, padding=True, truncation=True)

# Convert to tensor and move to device (if using GPU)
train_encodings = torch.tensor(train_encodings["input_ids"]).to(device)

# Fine-tune the GPT-2 model for text generation
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-5)  # Adjust learning rate as needed
model.train()  # Set model to training mode

epochs = 300
for epoch in range(epochs):
    # Access first element for labels (shifted by 1 for prediction)
    train_encodings["labels"] = train_encodings[0]
    outputs = model(**train_encodings)
    loss = outputs.loss
    loss.backward()
    optimizer.step()
    optimizer.zero_grad()
    print(f"Epoch {epoch+1}-Loss {loss.item()}")


#Save the fine-tuned model
output_dir="/content/fine_tuned_model_gpt2"
model.save_pretained(output_dir)
tokenizer.save_pretained(output_dir)

'''


'\nimport torch\nfrom transformers import GPT2Tokenizer, GPT2LMHeadModel\n\n#Load the pre-trained GPT-2 model and tokenizer\nmodel_name=\'gpt2\'\ntokenizer=GPT2Tokenizer.from_pretrained(model_name)\nmodel=GPT2LMHeadModel.from_pretrained(model_name)\n\n\nnum_params = sum(p.numel() for p in model.parameters())\nprint(f"Number of parameters in GPT-2 model: {num_params}")\n#124.39 Million\n\n\n#Set the device to GPU if available, else use CPU\ndevice=torch.device("cuda" if torch.cuda.is_available() else "cpu")\nmodel.to(device)\n\n#Text generation parameters\nprompt_text="I can do"\nmax_length=10\nnum_samples=1\n\n#Generate text samples\ngenerated_texts=[]\nfor _ in range(num_samples):\n  input_ids = tokenizer(prompt_text, return_tensors="pt").input_ids.to(device)\n  #input_ids = tokenizer.encode(prompt_text, return_tensor="pt").to(device)\n  with torch.no_grad():\n    output=model.generate(\n         input_ids,\n         max_length=max_length,\n         pad_token_id=tokenizer.eos_token_id

In [4]:
import pandas as pd

import torch
from torch.optim import AdamW
from torch.utils.data import Dataset, DataLoader, random_split, RandomSampler, SequentialSampler

from transformers import GPT2LMHeadModel, GPT2TokenizerFast, GPT2Config
from transformers import get_linear_schedule_with_warmup

from tqdm.auto import tqdm
import random
import datetime
import time

In [5]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"device {device}")

model_name = "gpt2"  # options: ['gpt2', 'gpt2-medium', 'gpt2-large', 'gpt2-xl']
model_save_path = '/content/model'

device cuda


In [6]:
configuration = GPT2Config.from_pretrained(model_name)
model = GPT2LMHeadModel.from_pretrained(model_name, config=configuration)

tokenizer = GPT2TokenizerFast.from_pretrained(model_name)

input_sequence = "beef, salt, pepper"
input_ids = tokenizer.encode(input_sequence, return_tensors='pt')

model = model.to(device)
#combine both sampling techniques
sample_outputs = model.generate(
                              input_ids.to(device),
                              do_sample = True,
                              max_length = 120,
                              top_k = 50,
                              top_p = 0.85,
                              num_return_sequences = 3
)

print("Output:\n" + 100 * '-')
for i, sample_output in enumerate(sample_outputs):
    print("{}: {}...".format(i, tokenizer.decode(sample_output, skip_special_tokens = True)))
    print('  ---')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token.As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


Output:
----------------------------------------------------------------------------------------------------
0: beef, salt, pepper, and olive oil.

In a large bowl, mix all of the butter and sugar. Bring to a boil, then reduce heat to low. Reduce heat to medium-low and simmer on low for 30 minutes.

Add eggs, olive oil, salt, and pepper. Stir well.

Let cool slightly. Add fresh lemon juice to the egg mixture and stir well. Cover and refrigerate for at least one hour....
  ---
1: beef, salt, pepper, pepper-and-salt, pepper, and honey. For added sweetness, add the onion and let it cook until it is soft and slightly charred. This is important, because onion is so easy to make, and it is so easy to store in the fridge.

To make this sauce: In a skillet, combine 2 tablespoons oil with 1 tablespoon butter. Add 1 teaspoon onion. Cook, stirring frequently until it is golden brown.

Add 2 teaspoons honey. Bring to a boil, add 1 teaspoon salt, and cook, stirring occasionally, until...
  ---
2: b

In [7]:
#Link to dataset:
#https://github.com/tuangatech/01-Recipes-Generation-GPT2/blob/main/recipes_1000.csv
#https://www.kaggle.com/datasets/irkaal/foodcom-recipes-and-reviews?select=recipes.csv

df_recipes = pd.read_csv('/content/recipes_1000.txt')
df_recipes.reset_index(drop=True, inplace=True)

# df_recipes = df_recipes.iloc[:600]
print(list(df_recipes.columns))
print(f"data shape {df_recipes.shape}")

['RecipeId', 'name', 'ingredients', 'instructions']
data shape (1000, 4)


In [8]:
import nltk
import numpy as np

import nltk
nltk.download('punkt')

doc_lengths = []

for rec in df_recipes.itertuples():

    # get rough token count distribution
    tokens = nltk.word_tokenize(rec.ingredients + ' ' + rec.instructions)

    doc_lengths.append(len(tokens))

doc_lengths = np.array(doc_lengths)
# the max token length
print(f"% documents > 180 tokens: {round(len(doc_lengths[doc_lengths > 180]) / len(doc_lengths) * 100, 1)}%")
print(f"Average document length: {int(np.average(doc_lengths))}")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


% documents > 180 tokens: 20.4%
Average document length: 133


In [9]:
def form_string(ingredient,instruction):
    # s = f"<|startoftext|>Ingredients:\n{ingredient.strip()}\n\nInstructions:\n{instruction.strip()}<|endoftext|>"
    s = f"<|startoftext|>Ingredients: {ingredient.strip()}. " \
        f"Instructions: {instruction.strip()}<|endoftext|>"
    return s

def extract_string(recipe):
    str = recipe.replace('<|startoftext|>', '').replace('<|endoftext|>', '')
    inst_pos = str.find('Instructions: ')
    ingredients = str[len('Ingredients: '): inst_pos-1]
    instructions = str[inst_pos+len('Instructions: '):]
    return ingredients, instructions

data = df_recipes.apply(lambda x:form_string(
    x['ingredients'], x['instructions']), axis=1).to_list()
data[0]


"<|startoftext|>Ingredients: blueberries, granulated sugar, vanilla yogurt, lemon juice. Instructions: Toss 2 cups berries with sugar. Let stand for 45 minutes, stirring occasionally. Transfer berry-sugar mixture to food processor. Add yogurt and process until smooth. Strain through fine sieve. Pour into baking pan (or transfer to ice cream maker and process according to manufacturers' directions). Freeze uncovered until edges are solid but centre is soft.  Transfer to processor and blend until smooth again. Return to pan and freeze until edges are solid. Transfer to processor and blend until smooth again. Fold in remaining 2 cups of blueberries. Pour into plastic mold and freeze overnight. Let soften slightly to serve.<|endoftext|>"

In [10]:
tokenizer = GPT2TokenizerFast.from_pretrained(model_name,
                                              bos_token='<|startoftext|>',
                                              eos_token='<|endoftext|>',
                                              unk_token='<|unknown|>',
                                              pad_token='<|pad|>'
                                             )

In [11]:
vocab_list = sorted(tokenizer.vocab.items(), key=lambda x:x[1])
for i in range(5555, 5566):
    print(vocab_list[i])

('ĠPhoto', 5555)
('Ġplus', 5556)
('rick', 5557)
('arks', 5558)
('Ġalternative', 5559)
('Ġpil', 5560)
('Ġapprox', 5561)
('that', 5562)
('Ġobjects', 5563)
('ĠRo', 5564)
('ĠAndroid', 5565)


In [12]:
print("The max model length is {} for this model".format(tokenizer.model_max_length))
print("The end of sequence token {} has the id {}".format(tokenizer.convert_ids_to_tokens(tokenizer.eos_token_id), tokenizer.eos_token_id))
print("The beginning of sequence token {} has the id {}".format(tokenizer.convert_ids_to_tokens(tokenizer.bos_token_id), tokenizer.bos_token_id))
print("The unknown token {} has the id {}".format(tokenizer.convert_ids_to_tokens(tokenizer.unk_token_id), tokenizer.unk_token_id))
print("The padding token {} has the id {}".format(tokenizer.convert_ids_to_tokens(tokenizer.pad_token_id), tokenizer.pad_token_id))

The max model length is 1024 for this model
The end of sequence token <|endoftext|> has the id 50256
The beginning of sequence token <|startoftext|> has the id 50257
The unknown token <|unknown|> has the id 50258
The padding token <|pad|> has the id 50259


In [13]:
# GPT2 is a large model. Increasing the batch size above 2 has lead to out of memory problems.
batch_size = 2
max_length = 180  # maximum sentence length

# standard PyTorch approach of loading data in using a Dataset class.
class RecipeDataset(Dataset):
    def __init__(self, data, tokenizer):
        self.data = data
        self.input_ids = []
        self.attn_masks = []
        self.origin_ingredients = []
        self.origin_instructions = []

        for recipe in data:
            encodings = tokenizer.encode_plus(recipe,
                                              truncation=True,
                                              padding='max_length',
                                              max_length=max_length,
                                              return_tensors='pt'       # return PyTorch tensor
                                             )
            self.input_ids.append(torch.squeeze(encodings['input_ids'],0))
            # attention_mask tells model not to incorporate these PAD tokens into its interpretation of the sentence
            self.attn_masks.append(torch.squeeze(encodings['attention_mask'],0))
            ingredients, instructions = extract_string(recipe)
            self.origin_ingredients.append(ingredients)
            self.origin_instructions.append(instructions)


    def __len__(self):
        return len(self.data)

    def __getitem__(self,idx):
        return self.input_ids[idx], self.attn_masks[idx], self.origin_ingredients[idx], self.origin_instructions[idx]

In [14]:
dataset = RecipeDataset(data, tokenizer)

# Split into training and validation sets
train_size = int(0.9 * len(dataset))
val_size = len(dataset) - train_size

train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

print('{:>5,} training samples'.format(train_size))
print('{:>5,} validation samples'.format(val_size))

  900 training samples
  100 validation samples


In [15]:
print(f"dataset size {dataset.__len__()}")
print(f"dataset[0]: \n  input_ids: {dataset[0][0]}\n  attn_masks: {dataset[0][1]}")

dataset size 1000
dataset[0]: 
  input_ids: tensor([50257, 41222,    25,  4171, 20853,    11, 19468,  4817,  7543,    11,
        16858, 32132,    11, 18873, 13135,    13, 27759,    25,   309,   793,
          362, 14180, 36322,   351,  7543,    13,  3914,  1302,   329,  4153,
         2431,    11, 26547, 10491,    13, 20558,   275,  6996,    12,    82,
        35652, 11710,   284,  2057, 12649,    13,  3060, 32132,   290,  1429,
         1566,  7209,    13,   520,  3201,   832,  3734,   264, 12311,    13,
        39128,   656, 16871,  3425,   357,   273,  4351,   284,  4771,  8566,
        16009,   290,  1429,  1864,   284, 11372,     6, 11678,   737, 34917,
        18838,  1566, 13015,   389,  4735,   475,  7372,   318,  2705,    13,
          220, 20558,   284, 12649,   290, 13516,  1566,  7209,   757,    13,
         8229,   284,  3425,   290, 16611,  1566, 13015,   389,  4735,    13,
        20558,   284, 12649,   290, 13516,  1566,  7209,   757,    13, 39957,
          287,  5637

In [16]:
# Create the DataLoaders for our training and validation datasets.
# We'll take training samples in random order.
train_dataloader = DataLoader(
            train_dataset,  # The training samples.
            sampler = RandomSampler(train_dataset), # Select batches randomly
            batch_size = batch_size # Trains with this batch size.
        )

# For validation the order doesn't matter, so we'll just read them sequentially.
validation_dataloader = DataLoader(
            val_dataset, # The validation samples.
            sampler = SequentialSampler(val_dataset), # Pull out batches sequentially.
            batch_size = batch_size # Evaluate with this batch size.
        )

## Fine-tuning LLM for Text Generation: GPT-2 model

In [17]:
configuration = GPT2Config.from_pretrained(model_name, output_hidden_states=False)
model = GPT2LMHeadModel.from_pretrained(model_name, config=configuration)
model = model.to(device)
print(f"Weight shape {model.transformer.wte.weight.shape}")
# this step is necessary because I've added some tokens (bos_token, etc.) to the embeddings
# otherwise the tokenizer and model tensors won't match up
model.resize_token_embeddings(len(tokenizer))
print(f"Number of tokens: {len(tokenizer)}")

# Set the seed value all over the place to make this reproducible.
seed_val = 42

random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

Weight shape torch.Size([50257, 768])
Number of tokens: 50260


In [18]:
word_embeddings = model.transformer.wte.weight # Word Token Embeddings

print(word_embeddings.shape)

torch.Size([50260, 768])


In [19]:
epochs = 3
learning_rate = 2e-5
warmup_steps = 1e2
# The epsilon parameter eps = 1e-8 is “a very small number to prevent any division by zero in the implementation”
epsilon = 1e-8
# optim = Adam(model.parameters(), lr=5e-5)
optim = AdamW(model.parameters(), lr = learning_rate, eps = epsilon)

def format_time(elapsed):
    return str(datetime.timedelta(seconds=int(round((elapsed)))))

In [20]:
# Total number of training steps is [number of batches] x [number of epochs].
# (Note that this is not the same as the number of training samples).
total_steps = len(train_dataloader) * epochs

# Create the learning rate scheduler.
# This changes the learning rate as the training loop progresses
scheduler = get_linear_schedule_with_warmup(optim,
                                            num_warmup_steps = warmup_steps,
                                            num_training_steps = total_steps)

In [21]:
def infer(prompt):
    input = f"<|startoftext|>Ingredients: {prompt.strip()}"
    input = tokenizer(input, return_tensors="pt")
    input_ids      = input["input_ids"]
    attention_mask = input["attention_mask"]

    output = model.generate(input_ids.to(device),
                            attention_mask=attention_mask.to(device),
                            max_new_tokens=max_length,
                            # temperature = 0.5,
                            do_sample = True, top_k = 50, top_p = 0.85)
                            # num_beams=5, no_repeat_ngram_size=2, early_stopping=True)
    output = tokenizer.decode(output[0], skip_special_tokens=True)
    return output

In [22]:
total_t0 = time.time()

training_stats = []

for epoch_i in range(0, epochs):
    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    print('Training...')

    t0 = time.time()

    total_train_loss = 0

    model.train()  # `train` just changes the *mode* (train vs. eval), it doesn't *perform* the training.

    for step, batch in enumerate(train_dataloader):     # step from enumerate() = number of batches

        b_input_ids = batch[0].to(device)   # tokens (of multiple documents in a batch)
        b_labels    = batch[0].to(device)
        b_masks     = batch[1].to(device)   # mask of [1] for a real word, [0] for a pad

        model.zero_grad()
        # loss = model(X.to(device), attention_mask=a.to(device), labels=X.to(device)).loss
        outputs = model(  input_ids = b_input_ids,
                          labels = b_labels,
                          attention_mask = b_masks,
                          token_type_ids = None
                        )

        loss = outputs[0]

        batch_loss = loss.item()
        total_train_loss += batch_loss

        # Get sample every x batches.
        if step % 100 == 0 and not step == 0:

            elapsed = format_time(time.time() - t0)
            print('  Batch {:>5,}  of  {:>5,}. Loss: {:>5,}.   Elapsed: {:}.'.format(step, len(train_dataloader), batch_loss, elapsed))

            model.eval()

            sample_output = infer("eggs, flour, butter, sugar")
            print(sample_output)

            # `train` just changes the *mode* (train vs. eval), it doesn't *perform* the training.
            model.train()

        loss.backward()
        optim.step()
        scheduler.step()

    # Calculate the average loss over all the batches.
    avg_train_loss = total_train_loss / len(train_dataloader)

    # Measure how long this epoch took.
    training_time = format_time(time.time() - t0)

    print("")
    print("  Average training loss: {0:.2f}".format(avg_train_loss))
    print("  Training epoch took: {:}".format(training_time))


    print("")
    print("Running Validation...")

    t0 = time.time()

    model.eval()

    total_eval_loss = 0
    nb_eval_steps = 0

    # Evaluate data for one epoch
    for batch in validation_dataloader:

        b_input_ids = batch[0].to(device)
        b_labels = batch[0].to(device)
        b_masks = batch[1].to(device)

        with torch.no_grad():

            outputs  = model(input_ids = b_input_ids,
                             attention_mask = b_masks,
                             labels = b_labels)

            loss = outputs[0]

        batch_loss = loss.item()
        total_eval_loss += batch_loss

    avg_val_loss = total_eval_loss / len(validation_dataloader)

    validation_time = format_time(time.time() - t0)

    print("  Validation Loss: {0:.2f}".format(avg_val_loss))
    print("  Validation took: {:}".format(validation_time))

    # Record all statistics from this epoch.
    training_stats.append(
        {
            'epoch': epoch_i + 1,
            'Training Loss': avg_train_loss,
            'Valid. Loss': avg_val_loss,
            'Training Time': training_time,
            'Validation Time': validation_time
        }
    )

print("")
print("Training complete!")
print("Total training took {:} (h:mm:ss)".format(format_time(time.time()-total_t0)))


======== Epoch 1 / 3 ========
Training...


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


  Batch   100  of    450. Loss: 3.6574506759643555.   Elapsed: 0:00:15.
Ingredients: eggs, flour, butter, sugar, brown sugar, cornmeal, baking powder, vanilla, cream, salt, salt, pepper, salt and pepperTo the mixer, cream, milk, vanilla, and vanilla extract(In large bowl, beat together flour, sugar, cream, butter, salt and pepper until combined. in bowl, blend in butter, vanilla, sugar, butter, egg, salt, cinnamon and vanilla. to mixer, cream, flour, sugar, mixture, and vanilla mixture until mixture is incorporated. combine in cooled, mixing bowl, and eggs. to whip cream mixture; eggs; mixture to whip powder,.1-inch dry, softened butter. in another bowl, mixer. beat the butter, cream, butter, sugar, and butter until cream mixture is smooth and peaks. Add in egg mixture;.2-inch baking powder, butter, cream and butter; eggs; beat until mixture is cooled.


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


  Batch   200  of    450. Loss: 2.2156524658203125.   Elapsed: 0:00:31.
Ingredients: eggs, flour, butter, sugar, baking powder, baking soda, cinnamon, vanilla extract, salt, pepper and black pepper.Preheat oven to 350 degreesF.Whisk together eggs, flour, sugar, baking powder, baking soda, cinnamon, vanilla extract, salt, pepper and black pepper. Melt butter in a small bowl. Add oil and melted butter mixture. Beat dough in butter until dough is well coated. Divide dough into 4 equal sized balls. Bake for about 20 minutes or until soft.Remove dough from pans and cool on paper until it cools.Stroll in flour and cooled dough. Divide dough into 2 equal sized balls.Roll out dough into 9 inch baking paper.Preheat oven to 350 degreesF. Brush a cookie sheet with the remaining baking soda. Bake in the oven for 20 minutes or until soft. Remove from pans and cool to room temperature.Cut into 2 inch pieces. Spread the mixture


...

  Batch   200  of    450. Loss: 1.5387649536132812.   Elapsed: 0:00:29.
Ingredients: eggs, flour, butter, sugar, salt, pepper, butter, sugar, salt, butter. Instructions: Combine all ingredients in small bowl; fold in eggs and flour. Fold in sugar; mix in flour mixture and pulse until stiffened; pour into prepared egg yolks; fill and fold in dry yolks. Bake at 350 for 20 minutes or until crisp.


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


  Batch   300  of    450. Loss: 2.678675413131714.   Elapsed: 0:00:44.
Ingredients: eggs, flour, butter, sugar, salt, baking soda, water. Instructions: Beat the flour and sugar together until fluffy. Add the flour mixture. Stir in the sugar and the eggs. Fold in the butter, salt, baking soda and water. Add the eggs and mix well. Stir in the egg mixture. Gradually mix in the remaining flour mixture. Gradually beat in the remaining butter and salt. Stir in the eggs and continue to stir until just blended. Serve hot.


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


  Batch   400  of    450. Loss: 1.3461605310440063.   Elapsed: 0:00:59.
Ingredients: eggs, flour, butter, sugar, milk, egg yolks, vinegar, salt, pepper, pepper. Instructions: Beat the butter and sugar in small batches. Add the flour and whisk together. Pour over the eggs, flour, sugar, salt, pepper, pepper and vinegar and stir well. Fold in the onion and squash. Gradually stir in the milk and wine until it has thickened. Gradually pour the milk into the hot water. Spread the sauce over the eggs and squash. Sprinkle with vinegar and the pepper. Bake at 350 degrees for 30 minutes, or until the skin is soft and slightly golden brown. Garnish with extra lettuce and a a slice of cucumber. Serve hot with fresh parsley.

  Average training loss: 2.11
  Training epoch took: 0:01:07

Running Validation...
  Validation Loss: 1.98
  Validation took: 0:00:02

Training complete!
Total training took 0:03:30 (h:mm:ss)


In [23]:
# Create a DataFrame from our training statistics.
df_stats = pd.DataFrame(data=training_stats)

# Use the 'epoch' as the row index.
df_stats = df_stats.set_index('epoch')
df_stats

,Training Loss,Valid. Loss,Training Time,Validation Time
epoch,,,,
1,6.104242,2.086312,0:01:10,0:00:02
2,2.185352,2.004826,0:01:07,0:00:02
3,2.113463,1.984221,0:01:07,0:00:02


## Saving & Loading Fine-Tuned Model

In [24]:
print("Saving model to %s" % model_save_path)

# Save a trained model, configuration and tokenizer using `save_pretrained()`.
# They can then be reloaded using `from_pretrained()`
# model_to_save = model.module if hasattr(model, 'module') else model  # Take care of distributed/parallel training
model.save_pretrained(model_save_path)
tokenizer.save_pretrained(model_save_path)

Saving model to /content/model


('/content/model/tokenizer_config.json',
 '/content/model/special_tokens_map.json',
 '/content/model/vocab.json',
 '/content/model/merges.txt',
 '/content/model/added_tokens.json',
 '/content/model/tokenizer.json')

In [25]:
model = GPT2LMHeadModel.from_pretrained(model_save_path)
tokenizer = GPT2TokenizerFast.from_pretrained(model_save_path)
model.to(device)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50260, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2SdpaAttention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50260, bias=False)
)

## Generate Text

In [26]:
# model = GPT2LMHeadModel.from_pretrained(model_save_path)
# tokenizer = GPT2TokenizerFast.from_pretrained(model_save_path)
# model.to(device)
print(infer("eggs, mushroom, butter, sugar"))

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Ingredients: eggs, mushroom, butter, sugar, lemon juice, salt, pepper, mustard, olive oil, black pepper, cumin, garlic powder. Instructions: In a large saucepan, combine egg, butter, sugar, lemon juice, salt, pepper, mustard, olive oil, cumin, garlic powder, garlic powder, garlic powder and saute for 5 minutes until well blended. Stir in salt, pepper, salt and pepper. Remove from heat and let cool to room temperature.


In [27]:
print(infer("onion, garlic, chicken breast"))

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Ingredients: onion, garlic, chicken breast, mushrooms, celery, oregano, salt, pepper, pepper flakes. Instructions: Mix together the onion, garlic, chicken breast, mushrooms, celery, ore. Pour over the onion and cook until cooked through. Drain, but not brown. Season with salt and pepper. Stir in the oregano, salt, pepper. Serve with the chicken and mushrooms. Instructions: In a medium saucepan, cook the chicken breasts on medium heat until tender. Add the onion, garlic, chicken, mushrooms, celery and oregano and cook until tender. Add the chicken and stir gently. Bring the sauce to a boil, reduce heat and simmer until the chicken is cooked through. When the sauce thickens, pour in the mushrooms. Stir over the chicken and stir into the mixture. Cover with the sauce and let stand until the mixture thickens. Transfer the mixture to a


In [28]:
print(infer("avocado, lime"))

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Ingredients: avocado, lime juice, water, cilantro, garlic, ginger, ginger, lime, salt. Instructions: Add cilantro, lime, lime juice, water, cumin, salt. Spread 1/2 cup of avocado over a lightly floured surface of a 9 x 13 inch baking dish or skillet. Cover and cook until thick and creamy. Set aside. Combine lime juice, lime juice, cilantro, garlic, ginger, lemon zest, lime peel, ginger, lime juice and salt. Spread on a baking sheet. Sprinkle with remaining  green onion, celery, chopped garlic, lemon zest, lime juice, cilantro, salt. Top with green onion and cover. Bake at 350 degrees for 15 to 20 minutes or until golden brown.


In [29]:
print(infer("beef, salt, pepper"))

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Ingredients: beef, salt, pepper, onions, tomato, garlic, celery, basil, cumin, salt, pepper. Instructions: Melt butter in a large saucepan and fry onion over medium heat. Add garlic. Cover and cook in uncovered heat. Cook for about 10 minutes or until translucent and translucent. Remove from heat and stir in the pepper. Place in a medium saucepan over medium heat. Simmer for about 30 seconds and then remove from heat. Stir in cumin. Stir in basil. Cook for a few minutes, stirring constantly. Remove from heat. Place in pot, add remaining beef, and cook for another 10 minutes. Stir in the tomatoes and bring to a simmer.


## Performance Evaluation with BLEU

In [30]:
# Using BLEU score to compare the real sentences with the generated ones
import statistics
from nltk.translate.bleu_score import sentence_bleu

scores=[]

for i in range(10):
    ingredients = val_dataset[i][2]
    reference = val_dataset[i][3]
    candidate = infer(ingredients)
    scores.append(sentence_bleu(reference, candidate))

print(statistics.mean(scores))

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consid

9.686658733279606e-232


# Fine-tuning LLM for Text Classification, Sentiment Analysis

In [31]:
from datasets import load_dataset, DatasetDict, Dataset

from transformers import (
    AutoTokenizer,
    AutoConfig,
    AutoModelForSequenceClassification,
    DataCollatorWithPadding,
    TrainingArguments,
    Trainer)

from peft import PeftModel, PeftConfig, get_peft_model, LoraConfig
import evaluate
import torch
import numpy as np

In [32]:
# how dataset was generated

# load imdb data
imdb_dataset = load_dataset("imdb")

# define subsample size
N = 1000
# generate indexes for random subsample
rand_idx = np.random.randint(24999, size=N)

# extract train and test data
x_train = imdb_dataset['train'][rand_idx]['text']
y_train = imdb_dataset['train'][rand_idx]['label']

x_test = imdb_dataset['test'][rand_idx]['text']
y_test = imdb_dataset['test'][rand_idx]['label']

# create new dataset
dataset = DatasetDict({'train':Dataset.from_dict({'label':y_train,'text':x_train}),
                              'validation':Dataset.from_dict({'label':y_test,'text':x_test})})

Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [33]:
# display % of training data with label=1
np.array(dataset['train']['label']).sum()/len(dataset['train']['label'])

0.501

In [34]:
model_checkpoint = 'distilbert-base-uncased'
# model_checkpoint = 'roberta-base' # you can alternatively use roberta-base but this model is bigger thus training will take longer

# define label maps
id2label = {0: "Negative", 1: "Positive"}
label2id = {"Negative":0, "Positive":1}

# generate classification model from model_checkpoint
model = AutoModelForSequenceClassification.from_pretrained(
    model_checkpoint, num_labels=2, id2label=id2label, label2id=label2id)

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [35]:
# display architecture
model

DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
 

In [36]:
# create tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, add_prefix_space=True)

# add pad token if none exists
if tokenizer.pad_token is None:
    tokenizer.add_special_tokens({'pad_token': '[PAD]'})
    model.resize_token_embeddings(len(tokenizer))

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [37]:
# create tokenize function
def tokenize_function(examples):
    # extract text
    text = examples["text"]

    #tokenize and truncate text
    tokenizer.truncation_side = "left"
    tokenized_inputs = tokenizer(
        text,
        return_tensors="np",
        truncation=True,
        max_length=512
    )

    return tokenized_inputs

In [38]:
# tokenize training and validation datasets
tokenized_dataset = dataset.map(tokenize_function, batched=True)
tokenized_dataset

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['label', 'text', 'input_ids', 'attention_mask'],
        num_rows: 1000
    })
    validation: Dataset({
        features: ['label', 'text', 'input_ids', 'attention_mask'],
        num_rows: 1000
    })
})

In [39]:
# create data collator
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [40]:
# import accuracy evaluation metric
accuracy = evaluate.load("accuracy")

In [41]:
# define an evaluation function to pass into trainer later
def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=1)

    return {"accuracy": accuracy.compute(predictions=predictions, references=labels)}

In [42]:
# define list of examples
text_list = ["It was good.", "Not a fan, don't recommed.", "Better than the first one.", "This is not worth watching even once.", "This one is a pass."]

print("Untrained model predictions:")
print("----------------------------")
for text in text_list:
    # tokenize text
    inputs = tokenizer.encode(text, return_tensors="pt")
    # compute logits
    logits = model(inputs).logits
    # convert logits to label
    predictions = torch.argmax(logits)

    print(text + " - " + id2label[predictions.tolist()])

Untrained model predictions:
----------------------------
It was good. - Negative
Not a fan, don't recommed. - Negative
Better than the first one. - Negative
This is not worth watching even once. - Negative
This one is a pass. - Negative


## Fine-tuning and Training

In [43]:
peft_config = LoraConfig(task_type="SEQ_CLS",
                        r=4,
                        lora_alpha=32,
                        lora_dropout=0.01,
                        target_modules = ['q_lin'])

In [44]:
peft_config

LoraConfig(peft_type=<PeftType.LORA: 'LORA'>, auto_mapping=None, base_model_name_or_path=None, revision=None, task_type='SEQ_CLS', inference_mode=False, r=4, target_modules={'q_lin'}, lora_alpha=32, lora_dropout=0.01, fan_in_fan_out=False, bias='none', use_rslora=False, modules_to_save=None, init_lora_weights=True, layers_to_transform=None, layers_pattern=None, rank_pattern={}, alpha_pattern={}, megatron_config=None, megatron_core='megatron.core', loftq_config={}, use_dora=False, layer_replication=None, runtime_config=LoraRuntimeConfig(ephemeral_gpu_offload=False))

In [45]:
model = get_peft_model(model, peft_config)
model.print_trainable_parameters()

trainable params: 628,994 || all params: 67,584,004 || trainable%: 0.9307


In [46]:
# hyperparameters
lr = 1e-3
batch_size = 16
num_epochs = 10

In [47]:
# define training arguments
training_args = TrainingArguments(
    output_dir= model_checkpoint + "-lora-text-classification",
    learning_rate=lr,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=num_epochs,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1494: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [48]:
# creater trainer object
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["validation"],
    tokenizer=tokenizer,
    data_collator=data_collator, # this will dynamically pad examples in each batch to be equal length
    compute_metrics=compute_metrics,
)

# train model
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.291399,{'accuracy': 0.877}
2,No log,0.278518,{'accuracy': 0.889}
3,No log,0.289131,{'accuracy': 0.884}
4,No log,0.399778,{'accuracy': 0.891}
5,No log,0.468666,{'accuracy': 0.88}
6,No log,0.572350,{'accuracy': 0.885}
7,No log,0.602202,{'accuracy': 0.881}
8,0.131400,0.627120,{'accuracy': 0.878}
9,0.131400,0.640503,{'accuracy': 0.879}
10,0.131400,0.650731,{'accuracy': 0.879}


Trainer is attempting to log a value of "{'accuracy': 0.877}" of type <class 'dict'> for key "eval/accuracy" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "{'accuracy': 0.889}" of type <class 'dict'> for key "eval/accuracy" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "{'accuracy': 0.884}" of type <class 'dict'> for key "eval/accuracy" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "{'accuracy': 0.891}" of type <class 'dict'> for key "eval/accuracy" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "{'accuracy': 0.88}" of type <class 'dict'> for key "eval/accuracy" as a scalar. This in

TrainOutput(global_step=630, training_loss=0.10638992899940128, metrics={'train_runtime': 517.4821, 'train_samples_per_second': 19.324, 'train_steps_per_second': 1.217, 'total_flos': 1326986724230400.0, 'train_loss': 0.10638992899940128, 'epoch': 10.0})

In [49]:
model.to('cpu') # you can alternatively do 'cpu' or 'cuda'

print("Trained model predictions:")
print("--------------------------")
for text in text_list:
    inputs = tokenizer.encode(text, return_tensors="pt").to("cpu") # you can alternatively do 'cpu' or 'cuda'

    logits = model(inputs).logits
    predictions = torch.max(logits,1).indices

    print(text + " - " + id2label[predictions.tolist()[0]])

Trained model predictions:
--------------------------
It was good. - Positive
Not a fan, don't recommed. - Negative
Better than the first one. - Positive
This is not worth watching even once. - Negative
This one is a pass. - Positive
